# A simple agent A2A Application with Custom Tools

This notebook presents a simple scenario where an agent uses the A2A protocol to query another agent for information using custom tools. We show how to initialize an agent in Llama Stack and grant it access to communicating with another, external agent.

This demo demonstrates core A2A communication principles.

## Overview

This notebook covers the following steps:

1. Setting up a Llama Stack agent with custom tool capabilities (e.g., random number generation, date retrieval).
2. Serving this agent over an A2A server.
3. Initializing another Llama Stack agent capable of communicating with the custom tool agent.
4. Launching the second agent and using it to answer user queries by leveraging the first agent's tools.

## Prerequisites

Before starting, ensure you have the following:
- `python_requires >= 3.13`

- Followed the instructions in the [Setup Guide](../../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb) notebook.

## Additional environment variables
This demo requires the following environment variables in addition to those defined in the [Setup Guide](../../rag_agentic/notebooks//Level0_getting_started_with_Llama_Stack.ipynb):
- `CUSTOM_TOOL_AGENT_LOCAL_PORT`: the port over which we will serve the exported A2A agent with custom tool capabilities.

## 1. Setting Up this Notebook
To provide A2A communication capabilities, we will use the [sample implementation by Google](https://github.com/google/A2A/tree/main/samples/python). Please make sure that the content of the referenced directory is available on your Python path. This can be done, for example, by running the following command:

In [1]:
! git clone https://github.com/google-a2a/a2a-samples.git
! pip install -r "../requirements.txt"

fatal: destination path 'a2a-samples' already exists and is not an empty directory.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Now, we will add the paths to the A2A library and our own tools to `sys.path`.

In [2]:
import sys
# the path of the A2A library
sys.path.append('./a2a-samples/samples/python')
# the path to our own utils
sys.path.append('../..')

We will now proceed with the necessary imports.

In [3]:
from common.server import A2AServer
from common.types import AgentCard, AgentSkill, AgentCapabilities
from a2a_llama_stack.A2ATool import A2ATool
from a2a_llama_stack.task_manager import AgentTaskManager

# for asynchronously serving the A2A agent
import threading

Next, we will initialize our environment as described in detail in our ["Getting Started" notebook](../../rag_agentic/notebooks/Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [4]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# agent related imports
import uuid
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger

# pretty print of the results returned from the model/agent - import from the rag_agentic demo subdirectory
import sys
sys.path.append('../../rag_agentic')  
from src.utils import step_printer
from termcolor import cprint


base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)
    
print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = os.getenv("INFERENCE_MODEL_ID")

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: llama3.1:8b-instruct-fp16
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 4096}
	stream: False


## 2. Setting Up and Serving a Simple A2A Agent with Custom Tools
We will now initialize an agent with custom tools (random number generator and date tool) and make it available via an A2A server.

Our first steps involve defining these custom tools and then creating an agent that knows how to use them.
- Define Python functions that will serve as our tools.
- Initialize a Llama Stack agent, providing it with these tools and instructions on how to use them.

In [5]:
import random
from datetime import datetime


def random_number_tool() -> int:
    """
    Generate a random integer between 1 and 100.
    """
    print("\n\nGenerating a random number...\n\n")
    return random.randint(1, 100)


def date_tool() -> str:
    """
    Return today's date in YYYY-MM-DD format.
    """
    return datetime.utcnow().date().isoformat()

- Initialize a Llama Stack agent with a list of tools including the built-in RAG tool. The RAG tool specification must include a list of document collection IDs to retrieve from.

In [6]:
custom_tool_agent = Agent(
    client,
    model=model_id,
    instructions=(
            "You have access to two tools:\n"
            "- random_number_tool: generates one random integer between 1 and 100\n"
            "- date_tool: returns today's date in YYYY-MM-DD format\n"
            "Always use the appropriate tool to answer user queries."
        ),    
    sampling_params=sampling_params,
    tools=[random_number_tool, date_tool],
    max_infer_iters=3,
)

Now, our Llama Stack agent is ready to be served as an A2A agent. This includes the following steps:
 - Create an `AgentCard` - an object containing all the details about the agent we are about to serve, including its URL and exposed capabilities.
 - Wrap the Llama Stack agent with an `AgentTaskManager` object - a wrapper/adapter making it possible for the A2A server to redirect incoming request to the Llama Stack agent.
 - Create and launch an `A2AServer` - a Rest API server capable of communicating via the A2A protocol.

In [ ]:
custom_tool_agent_local_port = int(os.getenv("CUSTOM_TOOL_AGENT_LOCAL_PORT", "10020"))
custom_tool_agent_url = f"http://localhost:{custom_tool_agent_local_port}"

agent_card = AgentCard(
    name="Custom Agent",
    description="Generates random numbers or retrieve today's dates",
    url=custom_tool_agent_url,
    version="0.1.0",
    defaultInputModes=["text/plain"],
    defaultOutputModes=["text/plain"],
    capabilities=AgentCapabilities(
        streaming=True,
        pushNotifications=False,
        stateTransitionHistory=False,
        ),
    skills=[
        AgentSkill(
            id="random_number_tool", 
            name="Random Number Generator",
            description="Generates a random number between 1 and 100",
            tags=["random"],
            examples=["Give me a random number between 1 and 100"],
            inputModes=["text/plain"],
            outputModes=["text/plain"],
            ),
        AgentSkill(
            id="date_tool",
            name="Date Provider",
            description="Returns today's date in YYYY-MM-DD format",
            tags=["date"],
            examples=["What's the date today?"],
            inputModes=["text/plain"],
            outputModes=["text/plain"],
            ),
    ],
)
task_manager = AgentTaskManager(agent=custom_tool_agent)
server = A2AServer(
    agent_card=agent_card,
    task_manager=task_manager,
    host='localhost',
    port=custom_tool_agent_local_port
)
thread = threading.Thread(target=server.start, daemon=True)
thread.start()

INFO:     Started server process [4581]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:10020 (Press CTRL+C to quit)


INFO:     127.0.0.1:49654 - "GET /.well-known/agent.json HTTP/1.1" 200 OK


/var/folders/p4/635191ns4599kwjkqt12kwd80000gn/T/ipykernel_4581/2943416845.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().date().isoformat()


INFO:     ::1:49667 - "POST / HTTP/1.1" 200 OK
INFO:     ::1:49696 - "POST / HTTP/1.1" 200 OK


## 3. Setting up an agent capable of A2A communication with the CUSTOM_TOOL agent
This includes the following steps:
 - Create a Llama Stack client tool that wraps A2A communication with the CUSTOM_TOOL agent.
 - Initialize a client agent with access to the above client tool.

In [8]:
custom_tool_agent_tool = A2ATool(custom_tool_agent_url)
a2a_client_agent = Agent(
    client,
    model=model_id,
    instructions="You are a helpful assistant. When a tool is used, only print its output without adding more content.",
    sampling_params=sampling_params,
    tools=[custom_tool_agent_tool],
)

Now, let's use our client agent for serving user requests.

In [9]:
queries = [
    "What is today's date?",
]

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # create a new turn with a new session ID for each prompt
    response = a2a_client_agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=a2a_client_agent.create_session(f"agent-session_{uuid.uuid4()}"),
        stream=stream,
    )
    
    # print the response, including tool calls output
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps)


User> What is today's date?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: Custom Agent, Arguments: {'query': "today's date"}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


'{\n    "type": "function",\n    "name": "date_tool",\n    "parameters": {}\n}Tool:date_tool Args:{}Tool:date_tool Response:"2025-06-03"{"type": "function", "name": "date_tool", "parameters": {}}Tool:date_tool Args:{}Tool:date_tool Response:"2025-06-03"I used the date_tool function to get today\'s date.'


---------- 📍 Step 3: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: Custom Agent, Arguments: {'query': "today's date"}

---------- 📍 Step 4: ToolExecutionStep ----------
🔧 Executing tool...


'{\n    "type": "function",\n    "name": "date_tool",\n    "parameters": {}\n}Tool:date_tool Args:{}Tool:date_tool Response:"2025-06-03"{"type": "function", "name": "date_tool", "parameters": {}}Tool:date_tool Args:{}Tool:date_tool Response:"2025-06-03"I used the date_tool function to get today\'s date.'


---------- 📍 Step 5: InferenceStep ----------
🤖 Model Response:
The current date is 2025-06-03.

========== Query processing completed ========== 

